In [216]:
import random
import numpy as np
import pandas as pd
from operator import add
import collections
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
import os
import pygame
import argparse
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from DQN import DQNAgent
from random import randint
import random
import statistics
import torch.optim as optim
import torch 
#from GPyOpt.methods import BayesianOptimization
#from bayesOpt import *
import datetime
import distutils.util
import json
import torchvision.transforms as transforms
import copy
from torch.utils.tensorboard import SummaryWriter
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE


device(type='cuda')

In [222]:
def check_game(array):
    ar = np.append(np.sum(array,1),np.sum(array,0))
    ar = np.append(ar,[sum(np.diag(array)),sum(np.diag(np.fliplr(array)))])
    if 3 in ar:
        return 1
    elif -3 in ar:
        return -1
    elif 0 not in array:
        return 0.5
    else:
        return 0
class Game():
    
    def __init__(self,xo,target = None):
        self.array = np.array([[0,0,0],[0,0,0],[0,0,0]])
        self.xo = xo
        #self.display = display
        if self.xo==-1:
            m = random.randint(0,2)
            n = random.randint(0,2)
            self.array[m][n] = 1
        self.done = 0
        """with open('q_table.txt') as f:
            q_table1 = f.read()"""
        #self.q_table1 = q_table1
        #print(q_table['10']['7'])
        #self.all_states1 = all_states1
        self.target = target
        if target == None:
            self.rand = True
        else:
            self.rand = False
        
    def update(self,ind):
        #print(ind)
        if self.array[(ind-1)//3][ind%3-1]==0:

            self.array[(ind-1)//3][ind%3-1] = self.xo

            if check_game(self.array)==0:
                if self.rand == False:
                    with torch.no_grad():

                        action = int(torch.argmax(self.target.forward(torch.FloatTensor(self.array,device=DEVICE))))+1
                    m = (action-1)//3
                    n = action%3-1
                else:
                    m = random.randint(0,2)
                    n = random.randint(0,2)

                while(self.array[m][n]!=0):
                    m = random.randint(0,2)
                    n = random.randint(0,2)
                self.array[m][n] = (self.xo*-1)
            self.result = check_game(self.array)
            if self.result!=0:
                self.done = 1
    def get_state(self):
        return self.array


In [223]:
def set_reward(xo, result):
    reward = 0
    if result == 1:
        if xo==1:
            reward = 10
        else:
            reward = -10
    elif result == -1:
        if xo == -1:
            reward = 10
        else:
            reward = -10
    elif result == 0.5:
        reward = 2
    return reward

In [224]:
num_episodes = 10000
max_steps_per_episode = 100
discount_rate = 0.99
exploration_rate = 0.9
max_exploration_rate = 0.9
min_exploration_rate = 0.05
exploration_decay_rate = 0.001
TAU = 0.005



In [225]:
""" def norm(x):
    x = np.expand_dims(x, axis=0)
    x_tensor = torch.from_numpy(x)
    normalize = transforms.Normalize(mean=[0.5], std=[0.5])
    x_normalized = normalize(x_tensor.float())
    x_normalized_np = x_normalized.numpy()
    x_normalized_np = np.squeeze(x_normalized_np, axis=0)
    final_x = torch.from_numpy(x_normalized_np)
    return final_x"""
class DQNAgent(torch.nn.Module):
    def __init__(self, params):
        super().__init__()
        self.reward = 0
        #self.gamma = 0.9
        """self.dataframe = pd.DataFrame()
        self.short_memory = np.array([])
        self.agent_target = 1
        self.agent_predict = 0"""
        self.learning_rate = params['learning_rate']  
        self.discount_rate = params['discount_rate']
              
        #self.epsilon = 1
        #self.actual = []
        self.batch_size = params['batch_size']
        self.first_layer = params['first_layer_size']
        self.second_layer = params['second_layer_size']
        self.third_layer = params['third_layer_size']
        self.memory = collections.deque(maxlen=params['memory_size'])
        self.weights = params['weights_path']
        #self.load_weights = params['load_weights']
        self.optimizer = None
        self.network()
        self.loss = 0
   
          
    def network(self):
        # Layers

        self.f1 = nn.Linear(9, self.first_layer)
        self.f2 = nn.Linear(self.first_layer, self.second_layer)
        #self.f3 = nn.Linear(self.second_layer, self.third_layer)
        self.f4 = nn.Linear(self.second_layer, 9)
        """# weights
        if self.load_weights:
            self.model = self.load_state_dict(torch.load(self.weights))
            print("weights loaded")"""

    def forward(self, x):
        #print(x)
        #z = x[0,1::3].clone()
        #z = z.int()
        #x = norm(x)
        #print(x)
        x = x.view( -1,9)
        x = F.relu(self.f1(x))
        x = F.relu(self.f2(x))
        #x = F.relu(self.f3(x))
        x = self.f4(x)
        #x[0][z==0]=-10000
        #x = F.softmax(x, dim=-1)
        #print(sum(x))
        return x
    def remember(self,tup):
        #print(tup)
        self.memory.append(tup)
    def update_agent(self,prev):
        if len(self.memory)>self.batch_size:

            batch = random.sample(self.memory,self.batch_size)
        else:
            return
        #print(batch)
        states, actions, rewards, next_states, dones = zip(*batch)
        states = torch.FloatTensor(states,device=DEVICE)
        actions = torch.LongTensor(actions,device=DEVICE)-1
        rewards = torch.FloatTensor(rewards,device=DEVICE)

        next_states = torch.FloatTensor(next_states,device=DEVICE)
        dones = torch.FloatTensor(dones,device=DEVICE)

        q_values = self.forward(states)
        #print(q_values)
        #print(actions)
        q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
        #print(next_states)
        next_q_values = prev(next_states).max(1)[0].detach()
        expected_q_values = rewards + self.discount_rate * next_q_values * (1 - dones)

        self.loss = nn.functional.smooth_l1_loss(q_values, expected_q_values)
        self.optimizer.zero_grad()
        self.loss.backward()
        self.optimizer.step()
        torch.nn.utils.clip_grad_value_(self.parameters(), 100)
        #print(len(batch),self.loss.item())
       

In [226]:
def define_parameters():
    params = dict()
    # Neural Network
    params['epsilon_decay_linear'] = 1/1000

    #params['learning_rate1'] = 0.1
    params['learning_rate'] = 0.0001
    params['discount_rate'] = 0.9
    params['first_layer_size'] = 27    # neurons in the first layer
    params['second_layer_size'] = 27 # neurons in the second layer
    params['third_layer_size'] = 54    # neurons in the third layer
    #params['episodes'] = 250          
    params['memory_size'] = 10000
    params['batch_size'] = 128
    # Settings
    params['weights_path'] = 'weights/weights.h5'
    #params['train'] = False
    #params["test"] = True
    #params['plot_score'] = True
    #params['log_path'] = 'logs/scores_' + str(datetime.datetime.now().strftime("%Y%m%d%H%M%S")) +'.txt'
    return params
#DEVICE = 'cpu'
params = define_parameters()
agent = DQNAgent(params)
agent = agent.to(DEVICE)
agent.optimizer = optim.Adam(agent.parameters(), weight_decay=0, lr=params['learning_rate'])
writer = SummaryWriter()
target_net = DQNAgent(params)
target_net = target_net.to(DEVICE)
target_net.load_state_dict(agent.state_dict())
target_net.eval()

rewards_all_episodes = []
"""with open('q_table.txt') as f:
    q_table = f.read()
    q_table = json.loads(q_table)
all_states = np.loadtxt('all_states.txt')"""
#all_states = np.array([Game(1).get_state()])
#q_table = dict()
#q_table[0] = {i:0 for i in range(1,10)}
k = 1
#q_table1 = None
#all_states1 = None
for episode in range(num_episodes):
    if (episode)%1000==0:
        #target_net.load_state_dict(agent.state_dict())
        print(episode)
        print(sum(rewards_all_episodes[-100:])/100)
        print(agent.loss)
    
    xo = [-1,1][random.randint(0,1)]
    game = Game(xo = xo,target= target_net)
    rewards_current_episode = 0
    t = 0
    for steps in range(max_steps_per_episode):
        state = np.copy(game.get_state())
        #print(state)
        if check_game(game.array)!=0:print(check_game(game.array))

        exploration_rate_threshold = random.uniform(0,1)
        if exploration_rate_threshold > exploration_rate:
            #print(game.array)
            with torch.no_grad():
                action = int(torch.argmax(agent.forward(torch.FloatTensor(state))))+1
                #print(game.array)
        else:
            action = random.randint(1,9)
        m = (action-1)//3
        n = action%3-1
        #print(m,n)
        #print(game.array)
        while(game.array[m][n]!=0):
            action = random.randint(1,9)
            m = (action-1)//3
            n = action%3-1
            #print(action)
        game.update(action)
        #print(game.array)
        next_state = np.copy(game.get_state())
        #print(state,action,next_state)
        reward = set_reward(xo,game.result)
        #print(state,next_state)
        agent.remember((state,action,reward,next_state,game.done))
        #print(agent.memory)
        agent.update_agent(target_net)
        #writer.add_scalar('Training Loss', agent.loss.item(), episode)
        #writer.add_scalar('Epsilon', exploration_rate, episode)
        #writer.add_scalar('Episode Length', t, episode)
        t+=1
        rewards_current_episode += reward
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = agent.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
        if game.done ==1:
            break
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate)*np.exp(-exploration_decay_rate*episode)
    rewards_all_episodes.append(rewards_current_episode)
    #print(sum(rewards_all_episodes[-10:])/10)
    #print(agent.memory)

        
            

0
0.0
0


RuntimeError: legacy constructor expects device type: cpu but device type: cuda was passed

In [199]:
np.array(rewards_all_episodes).reshape(10,100).sum(axis =1)

array([-124, -160,    4,   40,   52,  -36,  148,  100,   76,  -52])

In [208]:
agent.memory

deque([(array([[0, 0, 0],
               [0, 0, 0],
               [0, 0, 0]]),
        4,
        0,
        array([[ 0,  0,  0],
               [ 1, -1,  0],
               [ 0,  0,  0]]),
        0),
       (array([[ 0,  0,  0],
               [ 1, -1,  0],
               [ 0,  0,  0]]),
        1,
        0,
        array([[ 1,  0,  0],
               [ 1, -1,  0],
               [-1,  0,  0]]),
        0),
       (array([[ 1,  0,  0],
               [ 1, -1,  0],
               [-1,  0,  0]]),
        6,
        0,
        array([[ 1,  0,  0],
               [ 1, -1,  1],
               [-1,  0, -1]]),
        0),
       (array([[ 1,  0,  0],
               [ 1, -1,  1],
               [-1,  0, -1]]),
        3,
        -10,
        array([[ 1,  0,  1],
               [ 1, -1,  1],
               [-1, -1, -1]]),
        1),
       (array([[0, 0, 0],
               [0, 0, 0],
               [0, 0, 0]]),
        5,
        0,
        array([[ 0,  0,  0],
               [ 0,  1, -1],

In [148]:
import pickle
my_dict = pickle.load(open('data'))
my_dict

TypeError: a bytes-like object is required, not 'str'

In [150]:
with open('data') as f:
    print(f)

<_io.TextIOWrapper name='data' mode='r' encoding='cp1252'>


In [112]:
arr = np.array_split(rewards_all_episodes,10)
for i in arr:
    print(np.unique(i,return_counts=True))

(array([-10,   2,  10]), array([244,  63, 217], dtype=int64))
(array([-10,   2,  10]), array([197,  55, 272], dtype=int64))
(array([-10,   2,  10]), array([197,  87, 240], dtype=int64))
(array([-10,   2,  10]), array([178, 105, 241], dtype=int64))
(array([-10,   2,  10]), array([186, 159, 179], dtype=int64))
(array([-10,   2,  10]), array([205, 173, 145], dtype=int64))
(array([-10,   2,  10]), array([170, 200, 153], dtype=int64))
(array([-10,   2,  10]), array([178, 165, 180], dtype=int64))
(array([-10,   2,  10]), array([179,  65, 279], dtype=int64))
(array([-10,   2,  10]), array([173,  71, 279], dtype=int64))


In [191]:
z = 4
y = z
z = 3
x = z

In [206]:
class y():
    def __init__(self):
        self.d = np.array([1,2,3])
    def update(self,n):
        self.d[n] = 1
    def get(self):
        return self.d
Y = y()
x = Y.get()
Y.update(2)
s = Y.get()
x,s


(array([1, 2, 1]), array([1, 2, 1]))

In [144]:
q_values = torch.tensor([[0.1057, 0.1127, 0.1203, 0.0965, 0.1022, 0.1232, 0.1298, 0.1001, 0.1095],
        [0.1057, 0.1127, 0.1203, 0.0965, 0.1022, 0.1232, 0.1298, 0.1001, 0.1095]])
actions = torch.tensor([5,9])
print(actions-1)
q_values.gather(1, actions.unsqueeze(1))

tensor([4, 8])


RuntimeError: index 9 is out of bounds for dimension 1 with size 9

In [132]:
tup = (state,action,next_state,reward,game.done)
agent.memory.append(tup)
agent.memory

deque([(array([[ 0,  0,  0],
               [ 0,  1,  0],
               [ 0, -1,  1]]),
        8,
        array([[ 0,  0,  0],
               [ 0,  1,  0],
               [ 0, -1,  1]]),
        0,
        0),
       (array([[ 0,  0,  0],
               [ 0,  1,  0],
               [ 0, -1,  1]]),
        8,
        array([[ 0,  0,  0],
               [ 0,  1,  0],
               [ 0, -1,  1]]),
        0,
        0)],
      maxlen=2500)

In [135]:
batch = agent.memory
states, actions, rewards, next_states, dones = zip(*batch)
states = torch.FloatTensor(states)
actions = torch.LongTensor(actions)
rewards = torch.FloatTensor(rewards)
next_states = torch.FloatTensor(next_states)
dones = torch.FloatTensor(dones)

q_values = agent.forward(states)
print(q_values)
print(actions)
q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)

tensor([[[ 0.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0., -1.,  1.]],

        [[ 0.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0., -1.,  1.]]])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x18 and 9x200)

In [136]:
states.view(-1,9)

tensor([[ 0.,  0.,  0.,  0.,  1.,  0.,  0., -1.,  1.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0., -1.,  1.]])

In [129]:
q_values.gather(0,actions.unsqueeze(1))

RuntimeError: Index tensor must have the same number of dimensions as input tensor

In [86]:
def define_parameters():
    params = dict()
    # Neural Network
    params['epsilon_decay_linear'] = 1/1000

    #params['learning_rate1'] = 0.1
    params['learning_rate'] = 0.0001
    params['discount_rate'] = 0.001
    params['first_layer_size'] = 200    # neurons in the first layer
    params['second_layer_size'] = 20   # neurons in the second layer
    params['third_layer_size'] = 50    # neurons in the third layer
    params['episodes'] = 250          
    params['memory_size'] = 2500
    params['batch_size'] = 1000
    # Settings
    params['weights_path'] = 'weights/weights.h5'
    #params['train'] = False
    #params["test"] = True
    #params['plot_score'] = True
    #params['log_path'] = 'logs/scores_' + str(datetime.datetime.now().strftime("%Y%m%d%H%M%S")) +'.txt'
    return params
DEVICE = 'cpu'
params = define_parameters()
age = DQNAgent(params)
age.forward(torch.FloatTensor([[-1,1,0],[1,1,0],[-1,-1,0]]))


tensor([0.1045, 0.0953, 0.1050, 0.1141, 0.1088, 0.1321, 0.1053, 0.1159, 0.1190],
       grad_fn=<SoftmaxBackward0>)

In [128]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque

# Define the environment and the state space
env = gym.make('CartPole-v1')
state_space_size = env.observation_space.shape[0]

# Define the action space and the Q-function
action_space_size = env.action_space.n
q_function = nn.Sequential(
    nn.Linear(state_space_size, 32),
    nn.ReLU(),
    nn.Linear(32, action_space_size)
)

# Set hyperparameters
learning_rate = 0.001
discount_factor = 0.99
epsilon_start = 1.0
epsilon_end = 0.01
epsilon_decay = 0.999
batch_size = 32
memory_size = 100000

# Initialize the optimizer and the replay memory
optimizer = optim.Adam(q_function.parameters(), lr=learning_rate)
replay_memory = deque(maxlen=memory_size)

# Define a function to choose an action based on the epsilon-greedy policy
def choose_action(state, epsilon):
    if random.random() < epsilon:
        return env.action_space.sample()
    else:
        with torch.no_grad():
            q_values = q_function(torch.FloatTensor(state))
            return q_values.argmax().item()

# Define a function to update the Q-function using the Bellman equation and backpropagation
def update_q_function(batch):
    states, actions, rewards, next_states, dones = zip(*batch)
    states = torch.FloatTensor(states)
    actions = torch.LongTensor(actions)
    rewards = torch.FloatTensor(rewards)
    next_states = torch.FloatTensor(next_states)
    dones = torch.FloatTensor(dones)

    q_values = q_function(states)
    q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)

    next_q_values = q_function(next_states).max(1)[0].detach()
    expected_q_values = rewards + discount_factor * next_q_values * (1 - dones)

    loss = nn.functional.smooth_l1_loss(q_values, expected_q_values)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Train the Q-function
epsilon = epsilon_start
for episode in range(1000):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        action = choose_action(state, epsilon)
        p = env.step(action)
        print(p)
        break
        replay_memory.append((state, action, reward, next_state, done))
        total_reward += reward
        state = next_state

        if len(replay_memory) >= batch_size:
            batch = random.sample(replay_memory, batch_size)
            update_q_function(batch)

        if epsilon > epsilon_end:
            epsilon *= epsilon_decay
    break

    print(f"Episode {episode}: Total reward={total_reward}")


(array([ 0.04830463, -0.22097394,  0.04792751,  0.32241085], dtype=float32), 1.0, False, False, {})


In [81]:
agent = DQNAgent(params)
agent.optimizer = optim.Adam(agent.parameters(), weight_decay=0, lr=params['learning_rate'])
agent.optimizer.zero_grad()
nn.functional.smooth_l1_loss(torch.FloatTensor([4]),torch.FloatTensor([5])).backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [66]:
y = collections.deque(maxlen= 10)
y

deque([], maxlen=10)

In [67]:
for i in range(9):
    y.append(i)
y

deque([0, 1, 2, 3, 4, 5, 6, 7, 8], maxlen=10)

In [68]:
random.sample(y,20)

ValueError: Sample larger than population or is negative